---

## Description: Jupyter notebook to parse scanned .pdf documents
#### Author: Daniel Covarrubias
#### Date: 10OCT19
---


In [ ]:
#########################################
# Import necessary modules
#########################################

import pandas as pd
import numpy as np
import statsmodels
import torch
import os
import PyPDF2
import cv2
import imutils
import pytesseract
import sys
from PIL import Image
import platform
from pdf2image import convert_from_path
import matplotlib.pyplot as plt 
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn; seaborn.set() 
import pyodbc
# import tabula
## print(platform.architecture())

import pyocr
import pyocr.builders
# import wand
from wand.image import Image as Img
from wand.display import display
# import magickwand
# import textract

In [10]:
# s = os.environ["PATH"].split(';')
# for item in s:
#     print(item)

# pages = convert_from_path('//ushtfs01/data/Public/LPB_Projects/PURD (JUNO)/05 MSAT/11 SD_ER_GMP_data/PPQ/PPQ_1/Production/USHT-374 19H867007a.pdf', 500)

# with Img(filename='M://sandbox/Jupyter/batchRecords/input/USHT-374 19H867007a.pdf', resolution=300) as img:
#    img.compression_quality = 99
#    img.save(filename='image_name.jpg')

In [18]:
# # help('modules')
# # os.getcwd()

# # tools = pyocr.get_available_tools()
# # print(tools)
# # rawText = pytesseract.image_to_string(Image.open('M://sandbox/Jupyter/batchRecords/input/br_jpeg.jpeg'))

# #################################################################
# # that I used R to convert the .pdf file to a .jpeg file 
# #################################################################

# myImg = cv2.imread('M://sandbox/Jupyter/batchRecords/input/br_jpeg.jpeg', cv2.IMREAD_GRAYSCALE)
# cv2.imshow('image', myImg)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [11]:
# with Img(filename='//ushtfs01/data/Public/LPB_Projects/PURD (JUNO)/05 MSAT/11 SD_ER_GMP_data/PPQ/PPQ_1/Production/USHT-374 19H867007a.pdf', resolution=300) as img:
#     img.compression_quality = 99
#     img.save(filename='image_name.jpg')

In [19]:
# #########################################
# # Import pdf file
# #########################################

# # creating a pdf file object 
# pdfFileObj = open('//ushtfs01/data/Public/LPB_Projects/PURD (JUNO)/05 MSAT/11 SD_ER_GMP_data/PPQ/PPQ_1/Production/USHT-374 19H867007a.pdf', 'rb')

# # creating a pdf reader object 
# pdfReader = PyPDF2.PdfFileReader(pdfFileObj)

# # printing number of pages in pdf file 
# # print(pdfReader.numPages) 
  
# # creating a page object 
# pageObj = pdfReader.getPage(0) 
  
# # extracting text from page 
# print(pageObj.extractText()) 
  
# # closing the pdf file object 
# pdfFileObj.close() 

In [2]:
#######################################################
# Holy Smokes!  This is mostly working!!
# Incredible!
#######################################################

def angle_cos(p0, p1, p2):
    d1, d2 = (p0-p1).astype('float'), (p2-p1).astype('float')
    return abs( np.dot(d1, d2) / np.sqrt( np.dot(d1, d1)*np.dot(d2, d2) ) )

#######################################################

def find_squares(img, cos_limit = 0.1):
    print('search for squares with threshold %f' % cos_limit)
    img = cv2.GaussianBlur(img, (5, 5), 0)
    squares = []
    for gray in cv2.split(img):
        for thrs in range(0, 255, 26):
            if thrs == 0:
                bin = cv2.Canny(gray, 0, 50, apertureSize=5)
                bin = cv2.dilate(bin, None)
            else:
                retval, bin = cv2.threshold(gray, thrs, 255, cv2.THRESH_BINARY)
            contours, hierarchy = cv2.findContours(bin, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
            for cnt in contours:
                cnt_len = cv2.arcLength(cnt, True)
                cnt = cv2.approxPolyDP(cnt, 0.02*cnt_len, True)
                if len(cnt) == 4 and cv2.contourArea(cnt) > 1000 and cv2.isContourConvex(cnt):
                    cnt = cnt.reshape(-1, 2)
                    max_cos = np.max([angle_cos( cnt[i], cnt[(i+1) % 4], cnt[(i+2) % 4] ) for i in range(4)])
                    if max_cos < cos_limit :
                        squares.append(cnt)
                    else:
                        #print('dropped a square with max_cos %f' % max_cos)
                        pass
    return squares

In [2]:
# ##########################################
# # Define method() to identify squares
# ##########################################

# def find_squares(img):
#     ### cv2.split() is asking about input arguments
#     img = cv2.GaussianBlur(img, (5, 5), 0)
#     # img = cv2.GaussianBlur(img, ksize = 7)
#     squares = []
#     for gray in cv2.split(img):
#         for thrs in range(0, 255, 26):
#             if thrs == 0:
#                 bin = cv2.Canny(gray, 0, 50, apertureSize=5)
#                 bin = cv2.dilate(bin, None)
#             else:
#                 _retval, bin = cv2.threshold(gray, thrs, 255, cv2.THRESH_BINARY)
#             bin, contours, _hierarchy = cv2.findContours(bin, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
#             for cnt in contours:
#                 cnt_len = cv2.arcLength(cnt, True)
#                 cnt = cv2.approxPolyDP(cnt, 0.02*cnt_len, True)
#                 if len(cnt) == 4 and cv2.contourArea(cnt) > 1000 and cv2.isContourConvex(cnt):
#                     cnt = cnt.reshape(-1, 2)
#                     max_cos = np.max([angle_cos( cnt[i], cnt[(i+1) % 4], cnt[(i+2) % 4] ) for i in range(4)])
#                     #print(cnt)
#                     a = (cnt[1][1] - cnt[0][1])

#                     if max_cos < 0.1 and a < img.shape[0]*0.8:

#                         squares.append(cnt)
#     return squares

In [9]:
# #######################################################
# # Holy Smokes!  This is mostly working!!
# #######################################################

# def angle_cos(p0, p1, p2):
#     d1, d2 = (p0-p1).astype('float'), (p2-p1).astype('float')
#     return abs( np.dot(d1, d2) / np.sqrt( np.dot(d1, d1)*np.dot(d2, d2) ) )

# def find_squares(img):
#     img = cv2.GaussianBlur(img, (5, 5), 0)
#     squares = []
#     for gray in cv2.split(img):
#         for thrs in range(0, 255, 26):
#             if thrs == 0:
#                 bin = cv2.Canny(gray, 0, 50, apertureSize=5)
#                 bin = cv2.dilate(bin, None)
#             else:
#                 retval, bin = cv2.threshold(gray, thrs, 255, cv2.THRESH_BINARY)
#             contours, hierarchy = cv2.findContours(bin, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
#             for cnt in contours:
#                 cnt_len = cv2.arcLength(cnt, True)
#                 cnt = cv2.approxPolyDP(cnt, 0.02*cnt_len, True)
#                 if len(cnt) == 4 and cv2.contourArea(cnt) > 1000 and cv2.isContourConvex(cnt):
#                     cnt = cnt.reshape(-1, 2)
#                     max_cos = np.max([angle_cos( cnt[i], cnt[(i+1) % 4], cnt[(i+2) % 4] ) for i in range(4)])
#                     if max_cos < 0.1:
#                         squares.append(cnt)
#     return squares

In [13]:
# squares[6:10]
# tmp[8]
# cv2.contourArea(squares[1])

2247.5

In [3]:
# imgray = cv2.cvtColor(myImg, cv2.COLOR_BGR2GRAY)
# ret, thresh = cv2.threshold(imgray, 127, 255, 0)
# contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

#######################################################
# need to keep only smaller squares #
#######################################################

# find area of squares and keep those below threshold 

# (squares[1][1] - squares[1][0])[1] * 

#################################################
# Area of Square 
# first index needs to iterate 
#################################################

def find_area(squaresOutput):
    find_area_result = []
    for square in squaresOutput:
        # print(squaresOutput[k])
        # print(find_area_result)
        # find_area_result = abs((squaresOutput[k][1] - squaresOutput[k][0])[1] * (squaresOutput[k][3] - squaresOutput[k][0])[0])
        # tmp = abs((square[1] - square[0])[1] * (square[3] - square[0])[0])
        tmp = cv2.contourArea(square)
        find_area_result.append(tmp)
        # print(find_area_result)
    return find_area_result
# (squares[3][1] - squares[3][0])[1] * (squares[3][3] - squares[3][0])[0]

In [4]:
tmp = find_area(squares)

#################################################################
# Note that the squares are searched for in the cell below
#################################################################

plt.plot(tmp)
plt.show

# squares[6]
# plt.hist(tmp)

NameError: name 'squares' is not defined

In [5]:
########################################
# apply threshold logic #
# the threshold is currently set at 10
########################################

keep = pd.Series(np.array(tmp) < 10000)
# keep
# pd.DataFrame(np.array(tmp))
# np.array([keep, squares])
squares2 = np.array(squares)[keep]
# squares2
# np.array(squares)

NameError: name 'tmp' is not defined

In [ ]:
#######################################################
# Holy Smokes!  This is mostly working!!
# This is Awesome
#######################################################

# myImg = cv2.imread('M://sandbox/Jupyter/batchRecords/input/br_jpeg.jpeg')

myImg = cv2.imread('M://sandbox/Jupyter/batchRecords/input/br_png.png')
# np.size(cv2.split(myImg), 2)
# cv2.split(myImg)
# range(0,255,26)
squares = find_squares(myImg, cos_limit = 0.025)
cv2.drawContours(myImg, squares2, -1, (0, 255, 0), 3 )
cv2.imshow('squares', myImg)
cv2.waitKey(0)
cv2.destroyAllWindows()
# range(0, 255, 26)
# cv2.drawContours(myImg, squares, -1, (0, 255, 0), 3)

search for squares with threshold 0.025000


In [11]:
#######################################################
# Now Extract Images
#######################################################
image_number = 0

for square in squares2:
    area = cv2.contourArea(square)
    #if area > min_area and area < max_area:
    x,y,w,h = cv2.boundingRect(square)
    ROI = myImg[y:y+h, x:x+h]
    cv2.imwrite('./output/ROI_{}.png'.format(image_number), ROI)
    cv2.rectangle(myImg, (x, y), (x + w, y + h), (36,255,12), 2)
    image_number += 1

In [59]:
############################################
# run images through Tesseract #
############################################

text = pytesseract.image_to_string(Image.open('C://Users/Dcovarrubi/Documents/Jupyter/batchRecords/output/ROI_12.png'))

# tools = pyocr.get_available_tools()[0]

# text = tools.image_to_string(Image.open('C://Users/Dcovarrubi/Documents/Jupyter/batchRecords/output/ROI_11.png'),builder=pyocr.builders.DigitBuilder())

# img = Image.open('C://Users/Dcovarrubi/Documents/Jupyter/batchRecords/output/ROI_12.png')
# img = img.convert('L')
# img
# img.save('./output/testImage.jpg')

TesseractNotFoundError: tesseract is not installed or it's not in your path

In [179]:
# cnt = contours[4]
# cv2.drawContours(img, [cnt], 0, (0,255,0), 3)

In [235]:
# image = cv2.imread('M://sandbox/Jupyter/batchRecords/input/br_png.png')

# gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
# blur = cv2.medianBlur(gray, 5)
# sharpen_kernel = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]])
# sharpen = cv2.filter2D(blur, -1, sharpen_kernel)
    
# thresh = cv2.threshold(sharpen,160,255, cv2.THRESH_BINARY_INV)[1]
# kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
# close = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel, iterations=2)

# cnts = cv2.findContours(close, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
# cnts = cnts[0] if len(cnts) == 2 else cnts[1]

# min_area = 100
# max_area = 500
# image_number = 0
# for c in cnts:
#     area = cv2.contourArea(c)
#     if area > min_area and area < max_area:
#         x,y,w,h = cv2.boundingRect(c)
#         ROI = image[y:y+h, x:x+h]
#         cv2.imwrite('./output/ROI_{}.png'.format(image_number), ROI)
#         cv2.rectangle(image, (x, y), (x + w, y + h), (36,255,12), 2)
#         image_number += 1

# #cv2.imshow('sharpen', sharpen)
# #cv2.imshow('close', close)
# #cv2.imshow('thresh', thresh)
# cv2.imshow('image', image)
# # cv2.waitKey()
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [12]:
# cnts[2]